In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import glob
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from tqdm.notebook import tqdm_notebook as tq
import plotly.graph_objects as go


# Background

Covid-19 can possibly be the largest global distater of the decade with current number of reported cases standing at 217 Million. The effects are massive and not only on health but on daily lives. We have had to adapt to this norm as we can't let it stop us from living. Technology has been at the forefront in combating the spread of this virus, be it from medical research,global prediction and response equipment among many others. 

Education has been a major field of our lives that has been majorly hit. In the past most of the education involved in-person communication; i.e student and teacher are in the same physical location and space. Countries shut-down most educational institiuions at the begining of lockdown measures, as they aimed at limiting public gatherings and interactions
This by itself created a major setback on learning. Students had to adjust to learning using collaborative spaces and video-conferecning platforms. 

This has escaleted the ibalance in the education space, especially in how students access and the quality of education they do get. Factors such as socio-economic status, learning ability, envioronment all play a crucial role in determining the access to education.  

This analysis therefore aims at analysing this common factors that bring about inequality in the education system, and through that come up woth positive policies and solutions to how better not only governments but also schools can play their part in bridging the educational divide especially during this covid period 


# Data

The data used comes from LearnPlatform, an educational Technology company that aims to expand equitable access to education technology for all stdents and teachers. It generates evidence basis for what is currently working and enacting it to benefit of the parties involved.

The data collected comprised of 233 school districts toatllying 22 M datapoints to investiagte.
The data gives insights to educational/ learning platforoms used within the period of 1st January 2020 to 31st Decemeber 2020. This is was the begining and first and second wave of global spread, this generated lot's of activity on this platform

The dataset used is split into three categories
1. district_info.csv - which gives information about the educational district 
2. Products_info.csv -Which contains more description on the platforms used 
3. Engagement data -This is a folder that holds datasets that explain the enagement of the user and the varius platforms based on their distrcits



## Loading Data

In [ ]:
# #
products=pd.read_csv('../input/learnplatform-covid19-impact-on-digital-learning/products_info.csv')
districts=pd.read_csv('../input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv')


#to access all the engament datasets, we create a helper function to combine all of them into one huge dataset

for i in tq(range(1),desc="Testing"):
    pass

In [ ]:
home_dir=os.getcwd()

In [ ]:
#to access all the engament datasets, we create a helper function to combine all of them into one huge dataset
def get_all():
    try:
        os.chdir("../input/learnplatform-covid19-impact-on-digital-learning/engagement_data") 
        for i in tq(range(1),desc="Loading data") :
            folder='../input/learnplatform-covid19-impact-on-digital-learning/engagement_data/'
            extension='csv'
            all_files = [i for i in glob.glob('*.{}'.format(extension))]

            all_data=pd.concat([pd.read_csv(f) for f in all_files ])
#             print(all_files)

        os.chdir(home_dir)
        return all_data
        
    except Exception as e:
        
        print ("System error")
        sys.exit(1)

In [ ]:
data=get_all()

Lets have a look at all the available data

In [ ]:
data

In [ ]:
data.shape

We have merged the engagement data into a single csv. It contains 22324190 records and four features.

In [ ]:
products

The products column includes 372 entries and 6 features

In [ ]:
districts

The districts column has 233 records with 7 feature columns

# Exploratory Data Analysis

## District data

In [ ]:
#we define a couple of scripts that we shall implement
def null_values(df):
    
    #get the number of null values in the system
    try:
        print(" #### Calculating Missing Values ##### \n")
        for i in tq(range(1),desc="Fetching data"):
        
            
            
            sumofNull=df.isna().sum()
            percentage=sumofNull/len(df)*100
        print(" #### Done #### \n ")
            
            
        
        print("#### Creating data frame #### \n ")
        
        for i in tq(range(1),desc="Creating DataFrame"):
            valuesdf=pd.DataFrame(data=[sumofNull,percentage])
            valuesdf=valuesdf.T
            valuesdf.columns=["Total Missing","Percentage Missing"]
        print(" #### Done #### ")
        

        return valuesdf
            
    except Exception as e:
        
        print(" !!! Error File Not Found  !!!!! \n")
        print(" !!! Program Failed !!!!! \n")
        
        print("Safely exiting the program")
        sys.exit(1)

def dropDuplicates(data):
        try:
            
            for i in tq(range(1),desc="Detecting duplicates"):
                data=data.drop_duplicates()
            for i in tq(range(1),desc="Report on Duplicates "):
                dupCount=len(data)-len(data.drop_duplicates())
                
            print ("There are {} duplicates in the dataset\n".format(dupCount))
            logging.info("Number of duplicates in the datset are {} ".format(dupCount))

            

            return data 
        except Exception as e:

            print("The following error occured {} ".format(e.__class__))

## Districts

**Columns**
The data features in the dataset are 
1. district_id : The respective district identification number
2. State : The state that the district belongs to 
3. locale : The locale the given district is located
4. pc_black/hispanic : The percntage of students that identify as black/hispanic
5. pct_free/reduced : percentage of students eliglble for free or reduced lunch
6. county_connections_ratio : ratio of high internet speeds 
7. pp_total_raw : sum of local and federal expenditure per pupil 

### Understanding the Missing Values

In [ ]:
districts

In [ ]:
districts.info()

From the quick summary above we can tell we have quite a number of null values. From the description the null values are meant to create a form of anonymity,hence most of the data is structurally missing.

Lets calculate the missing values

In [ ]:
null_values(districts)

We can discover that if the state data is missing then locale and pc_black/hispanic records are probably also missing.
We see that ppl_total_raw has roughly 49% null values which isn't that good for our anlysis

**Lets deal with values that are missing from both state and locale**

This is because we can not examine a state/locale we don't know 

In [ ]:
test=districts[(districts['state'].notnull())& (districts['locale'].notnull())]

In [ ]:
test

In [ ]:
null_values(test)

The reamining values seem to be a small percentage of the data and we can hence use techniques to impute this values, we can handle the missing values through. We drop rows that all the values are null and see if we can narrow down on our null values 

In [ ]:
test

In [ ]:
nulls=test[(test['pct_black/hispanic'].isnull()) | (test['pct_free/reduced'].isnull())]

nulls

We stumble upon an interesting finding most of the null values in the pct_free/reduced column are from Massachusetts, is it that the whole state didn't record it or they actually don't have that lets find out 

In [ ]:
data_massachets=test[test['state']=='Massachusetts']
data_massachets

Every district under the state of Massachusetts didn't record any values in the amount of percentage of students eliglble for free or reduced lunch. Lets investiagte the web to analyze the findings...

We find out thet Massachusetts do have a free lunch policy in their state and has been implementing it since 1984, google stats shows that the state offers free lunch to 51% of their students.
Hence we can fill in the missing columns with between 40 and 50 %

In [ ]:
test['pct_free/reduced'].fillna('[0.4, 0.6[',inplace=True)
test

In [ ]:
null_values(test)

We now investiagte the percentage of expenidture per pupil in the missing values

In [ ]:
pct_null=test[test['pp_total_raw'].isnull()]
pct_null

A couple of states are missing this column and hence lets dive deep and understand whether there is any relationship between this and missing values

In [ ]:
test[test['state']=='Connecticut']

We also discover that all Connecticut states have null values in their percentage expenidure per pupil, we can investigate more on the internet

From this article [here](http://https://yankeeinstitute.org/2020/05/18/connecticut-has-third-highest-spending-per-public-school-student-in-the-country/) Connecticut spends $20,635 on per pupil expenditure so we impute those for the missing values

In [ ]:
# r=test[(test['state']=='Connecticut')&(test['pp_total_raw'].isnull())]['pp_total_raw'].fillna('[14000, 16000[')
# r
test['pp_total_raw']=np.where(test['state']=='Connecticut','[14000, 16000[' , test.	pp_total_raw)
test

In [ ]:
null_values(test)

We are making progress huh?

In [ ]:
test[test['state']=='Connecticut']

In [ ]:
test[test['state']=='California']

As with Connecticut we experience the same issue with California and hence we head back to our article to find answers
and we do find out that it has $12,498 per pupil spending expenditure

Hence we impute the missing values with the interval close to that

In [ ]:
test['pp_total_raw']=np.where(test['state']=='California','[11000, 13000[' , test.pp_total_raw)
test

In [ ]:
null_values(test)

## Understanding States

In [ ]:
#we get the state abbrevations to use in potting graphs
state_abb = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District Of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

districts['state_abb'] = districts['state'].map(state_abb)




**Unique Values**

In [ ]:
def count_plot(data,colr,title):
    plt.figure(figsize=(10,8))
    ax=sns.countplot(x=data,palette=colr,order=data.value_counts().index)
    plt.xticks(rotation=90)
    plt.title(title)
    for p in ax.patches:
        ax.text (p.get_x() + p.get_width()  / 2,p.get_height()+ 0.75,p.get_height(), fontsize = 11)
#         ax.text('%{:.1f}'.format(p.get_height()), (p.get_x()+0.1, p.get_height()+50))
def count_plot_xy(data,colr,title,data2):
    plt.figure(figsize=(10,8))
    ax=sns.countplot(x=data,y=data2,palette=colr,order=data.value_counts().index)
    plt.xticks(rotation=90)
    plt.title(title)
    for p in ax.patches:
        ax.text (p.get_x() + p.get_width()  / 2,p.get_height()+ 0.75,p.get_height(), fontsize = 11)
#         ax.text('%{:.1f}'.format(p.get_height()), (p.get_x()+0.1, p.get_height()+50))
def geo_map(title,state_abb,colorscheme):
    fig = go.Figure()
    layout = dict(title_text = title,title_font = dict(family = "monospace",size = 25,color = "black"),geo_scope = 'usa')

    fig.add_trace(go.Choropleth(locations = state_abb.value_counts().to_frame().reset_index()['index'],
            zmax = 1,
            z = state_abb.value_counts().to_frame().reset_index()['state_abb'],
            locationmode = 'USA-states',
            marker_line_color = 'white',
            geo = 'geo',
            colorscale = colorscheme, 
        )
    )

    fig.update_layout(layout)   
    fig.show()


In [ ]:
geo_map("Count of districts in the available States",districts['state_abb'],"dense")

In [ ]:
# fig = go.Figure()
# layout = dict(
#     title_text = "Count of districts in the available States",
#     title_font = dict(
#             family = "monospace",
#             size = 25,
#             color = "black"
#             ),
#     geo_scope = 'usa'
# )

# fig.add_trace(
#     go.Choropleth(
#         locations = districts['state_abb'].value_counts().to_frame().reset_index()['index'],
#         zmax = 1,
#         z = districts['state_abb'].value_counts().to_frame().reset_index()['state_abb'],
#         locationmode = 'USA-states',
#         marker_line_color = 'white',
#         geo = 'geo',
#         colorscale = "cividis", 
#     )
# )
            
# fig.update_layout(layout)   
# fig.show()


In [ ]:
count_plot(districts['state'],'RdYlGn',"State representation")

Connecticut has the most number of district representation with 30 district counts in the dataset closely followed by Utah

In [ ]:
plt.figure(figsize = (15, 8))
sns.set_style("white")
a = sns.barplot(data = districts['state'].value_counts().reset_index(), x = 'state', y = 'index', color = '#90afc5')
plt.xticks([])
plt.yticks(fontsize = 14, color = '#283655')
plt.ylabel('')
plt.xlabel('')

a.spines['left'].set_linewidth(1.5)
for w in ['right', 'top', 'bottom']:
    a.spines[w].set_visible(False)
    
for p in a.patches:
    width = p.get_width()
    plt.text(0.5 + width, p.get_y() + 0.55 * p.get_height(), f'{int(width)}',
             ha = 'center', va = 'center',  fontsize = 15, color = '#283655')

plt.show()

## Locales distribution in districts

In [ ]:
count_plot(districts['locale'],'Blues','locale representation') 

A great number of the education insituions are located in the Suburbs 
**but does this result in bettter education 
How do they compare to the other locales**?

We investigate the number of locality found in different states

In [ ]:
plt.figure(figsize=(20,10))
ax=sns.countplot(data=districts,x='state',hue='locale',palette='Blues')
plt.xticks(rotation=90)
plt.title("State vs Locality")
for p in ax.patches:
    ax.text (p.get_x() + p.get_width()  / 2,p.get_height()+ 0.75,p.get_height(), fontsize = 11)


1. We find that most of the states expect califronia are made up of the suburbs,
2. california is made up of mainly tow locales, Arizona only has twin locales 
3. NorthDakota has Rural locales 
4. New Hempshire as well
5. Tennessee is made up of rural and town locales

### Fixing the interval columns

We aggeregate by state and find the percentages features. But first a quick look at the column we do realize that the column is made of intervals.

A quick intro to interval notation:

***]a,b[ := {: a<x <b }*** : **open Real interval**

***[a,b[ := {a<= x <b}***  :  **Half-open on the right**

***]a,b] := {a<xb<=b}*** : **Half-open on the left**

we address this by spliting by the , and taking the values 

We however have to impute the values first before carrying on with the analysis


In [ ]:
distcrict_copy=districts.copy()
districts.dropna(inplace=True)

In [ ]:

districts['pct_black/hispanic']=districts['pct_black/hispanic'].apply(lambda x :float(x.split(',')[0][1:])+0.1)

districts['pct_free/reduced']=districts['pct_free/reduced'].apply(lambda x :float(x.split(',')[0][1:])+0.1)

In [ ]:
districts=districts.reset_index()

In [ ]:
districts.drop(labels='index',inplace=True,axis=1)

In [ ]:
districts

In [ ]:
districts['pp_total_raw']=districts['pp_total_raw'].apply(lambda x :float(x.split(',')[0][1:])+1000)
districts['county_connections_ratio']=districts['county_connections_ratio'].apply(lambda x: float(x.split(',')[0][1:])+0.1)

districts


We aggregate the data by state and hopeully understand what goes on there

In [ ]:
state_locale_df=districts.groupby(['state_abb','locale']).agg({'pct_black/hispanic':np.mean,'pct_free/reduced':np.mean,'pp_total_raw':np.mean})
state_locale_df=state_locale_df.reset_index()
state_locale_df

## Pct_Black and Hispanics

In [ ]:
state_pct=districts.groupby('state').agg({'pct_black/hispanic':np.mean,'pct_free/reduced':np.mean,'pp_total_raw':np.mean})

In [ ]:
state_pct=state_pct.reset_index()

In [ ]:
state_pct['state_abb']=state_pct['state'].map(state_abb)

In [ ]:
#is there a relationship between the two ratios?
sns.heatmap(state_pct.corr(),annot=True)

In [ ]:
fig = px.bar_polar(state_pct, r="pct_black/hispanic", theta="state", 
            color_discrete_sequence= px.colors.sequential.thermal)
fig.show()

Texas has the highest number of black/hispanic representation
Quick review of the polar chart we spot that it doesn't have any clean relationship between pct_black/hispanic and pct_free/reduced 

New york seems to have the highest number of expenditure per pupil, we also do notice that the states that had higher pct_ofblacks/hispanic has a lower expenditure per pupil

Can this be affected by the locale of a district , lets viusialize and see if it affects that

In [ ]:
# sample funnel graph showing the percentage of black/hispanic students in each state

fig = px.funnel(state_pct, x='pct_black/hispanic', y='state')
fig.show()

### Percentage of Blacks and Hispanic

In [ ]:
fig = px.bar_polar(state_pct, r="pct_free/reduced", theta="state", template="plotly_dark",
            color_discrete_sequence= px.colors.sequential.Plasma_r)
fig.show()

In [ ]:
#create a locale 
locale_df=districts.groupby('locale').agg({'pct_black/hispanic':np.mean,'pct_free/reduced':np.mean})

In [ ]:
locale_df=locale_df.reset_index()

### Percentage Free Lunch

In [ ]:
plt.figure(figsize=(15,9))
ax=sns.barplot(x="locale", y="pct_black/hispanic", data=state_locale_df,palette="magma")

plt.xticks(rotation=90)
plt.title("Percentage of blacks and hispanic in the locales")






We notice that cities have the highest percentage of black/hispanic students while rural locales have the least percentage.
Utah also has the highest percentage but can this be due to the high representation of districts in the datset

Highest percentage of blacks and hispanics can be found in City with towns having the least number of black/ hispanic representation

In [ ]:

def geo_map_z(title,loc,param,col):
    
    fig = go.Figure()
    layout = dict(
        title_text = title,
        title_font = dict(family = "monospace",size = 25,color = "black"),geo_scope = 'usa') 

    fig.add_trace(
        go.Choropleth(
            locations = loc.value_counts().to_frame().reset_index()['index'],
            zmax = 1,
            z = param,
            locationmode = 'USA-states',
            marker_line_color = 'white',
            geo = 'geo',
            colorscale = col, 
        )
        
    )

    fig.update_layout(layout)   
    fig.show()

# geo_map("Count of districts in the available States",districts['state_abb'],"dense")
geo_map_z("Percentage of blacks per state",state_locale_df['state_abb'],state_locale_df['pct_black/hispanic'],"picnic")

In [ ]:
geo_map_z("Free or reduced lunch in State",state_locale_df['state_abb'],state_locale_df['pct_free/reduced'],"Reds")

In [ ]:
plt.figure(figsize = (15, 8))
sns.set_style("white")
a = sns.barplot(data = state_pct, y = 'state', x = 'pct_free/reduced', color = '#90afc5')
# plt.xticks([])
plt.yticks(fontsize = 14, color = '#283655')
plt.ylabel('Pct_free/Reduced fee')
plt.xlabel('State')

a.spines['left'].set_linewidth(1.5)
for w in ['right', 'top', 'bottom']:
    a.spines[w].set_visible(False)
    
    
# for p in a.patches:
#     ax.text (p.get_x() + p.get_width()  / 2,p.get_height()+ 0.75,p.get_height(), fontsize = 11)

plt.show()

In [ ]:
plt.figure(figsize=(15,15))
sns.catplot(y="pct_free/reduced", x="state",hue="locale",palette="rocket",data=districts, kind="bar",height=7,aspect=2)
plt.xticks(rotation=90);
plt.title("Distribution of state percentage of reduced and free lunches")

In [ ]:
# fig = go.Figure()
# layout = dict(
#         title_text = "test",
#         title_font = dict(family = "monospace",size = 25,color = "black"),geo_scope = 'usa') 

# fig.add_trace(
#         go.Choropleth(
#             locations = loc.value_counts().to_frame().reset_index()['index'],
#             zmax = 1,
#             z = param,
#             locationmode = 'USA-states',
#             marker_line_color = 'white',
#             geo = 'geo',
#             colorscale = col, 
#         )
        
#     )

# fig.update_layout(layout)   
# fig.show()

Most of the states mainly consist of Suburb locales. This locales seem to have the highest number of pct of hispanic and blacks except in Illinois and North Carolina. 

### Expenditure used per pupil

In [ ]:
# fig = go.Figure()
# layout = dict(
#     title_text = "Percentage reduced fees per state",
#     title_font = dict(
#             family = "monospace",
#             size = 25,
#             color = "black"
#             ),
#     geo_scope = 'usa'
# )

# fig.add_trace(
#     go.Choropleth(
#         locations = state_pct['state_abb'].value_counts().to_frame().reset_index()['index'],
#         zmax = 1,
#         z = state_pct['pct_free/reduced'],
#         locationmode = 'USA-states',
#         marker_line_color = 'white',
#         geo = 'geo',
#         colorscale = "peach", 
#     )
# )
            
# fig.update_layout(layout)   
# fig.show()


In [ ]:
fig = px.bar_polar(state_pct, r="pp_total_raw", theta="state", template="plotly_dark",
            color_discrete_sequence= px.colors.sequential.Plasma_r)
fig.show()

As we had earlier read in the article New York has the highest number of per pupil expenditure and New-Jersey close by

In [ ]:
state_locale_df

In [ ]:
plt.figure(figsize=(15,15))
sns.catplot(y="pp_total_raw", x="state_abb",hue="locale",palette="rocket",data=state_locale_df, kind="bar",height=7,aspect=2)
plt.xticks(rotation=90);
plt.title("Distribution of state percentage of reduced and free lunches")

New Jersey and and New York only have Suburbs and Rural settings can this be a confounder in the amount of expenditure used by governements

In [ ]:
plt.figure(figsize=(15,9))
ax=sns.barplot(x="locale", y="pp_total_raw", data=state_locale_df,palette="magma")

plt.xticks(rotation=90)
plt.title("Percentage of blacks and hispanic in the locales")

Localities in the Suburbs and rural areas generally Suburbs and Rural locales

In [ ]:
#Does having a increasing percentage of free lunch students affect expenditure per students?
fig = px.bar_polar(state_locale_df, color="pct_free/reduced", theta="state_abb", r="pp_total_raw",template="plotly_dark",
            color_discrete_sequence= px.colors.sequential.Plasma_r)
fig.show()

We look at the distribution of locales per area

## Questions that arise

1. Black/Hispanic representation in Texas, North Cal and Florida
2. Why city locales have the highest number of black/hispanic rep
3. Do locales affect percentage fees reduced
4. Does having higher percentage of free/reduced lunch affect the amount expenditure per pupil


## products

In [ ]:
products

From the look of things we do have lots of products here,lets start of by undetsanding what each column contains

1. **LP_ID-** Unique product identifier 
2. **URL**-Url of the product
3. **Product Name** - Name of the product
4. **Provider/ Company Name** - Owner of the product
5. **Sector(s)**- Sector the product falls under
6. **Primary Essential Function**- The main purpose the product serves: LC= learning Curriculum, CM=Classroom Managment SDO= School & District Operations

In [ ]:
products.shape

The dataset has 372 records and 6 features

In [ ]:
products.info()

## EDA

In [ ]:
products

## Understanding the Nulls

In [ ]:
null_values(products)

Most of the missing values appear in the sectors and primary Essential Functions.. My intuition tells me that if one is missing the other is also missing..

Lest find out


In [ ]:
products[products['Sector(s)'].isna()]

Yup thought so, so we could actually run through the null values and see what we can use to impute them since some of the products are commonly used platforms

### Sector

In [ ]:
products['Sector(s)'].value_counts()

Most of the missing values are platforms that are used for multipurpose as they fit in all the sectors. Hence impute using new multiPurpose entry

In [ ]:
products['Sector(s)']=np.where(products['Sector(s)'].isna(),'Multipurpose',products['Sector(s)'])
products

In [ ]:
products['Primary Essential Function'].value_counts()

We create a new essential function called MT=Multipurpose this is for the new imputed values 

In [ ]:
products['Primary Essential Function']=np.where(products['Primary Essential Function'].isna(),'GP - General Purpose',products['Primary Essential Function'])
products

In [ ]:
null_values(products)

We have one more null value which is in the provider/Company Name so lets resolve that


In [ ]:
products[products['Provider/Company Name'].isna()]

A quick google search can provide the parent company of True North Logic, which happens to be PowerSchool Group LLC so we impute the missing value there

In [ ]:
products['Provider/Company Name']=np.where(products['Provider/Company Name'].isna(),'PowerSchool Group LLC',products['Provider/Company Name'])
products

In [ ]:
null_values(products)

Finally Done lets continue shall we...

With that done lets do some pre-processing and feature extraction to the data

## Pre Processing

In [ ]:
products['Function Class']=products['Primary Essential Function'].apply(lambda x : x.split('-')[0])
products['Function Desc']=products['Primary Essential Function'].apply(lambda x : x.split('-')[1])

products

We can then drop the Primary Essential Function column since we have already extracted the features we require

In [ ]:
products.drop(labels=['Primary Essential Function'],axis=1,inplace=True)
products

In [ ]:
products.info()

## Visaulize

### Products

Whats the most Used products ?

In [ ]:
products['Product Name'].value_counts()

Most of the products are just singles so we move and see if any company owns mulitple services

In [ ]:
#we get the top ten provider companies 
products['Provider/Company Name'].value_counts()[:10].index

In [ ]:

plt.figure(figsize=(12,8))
ax=sns.countplot(products['Provider/Company Name'],palette='cool_r',order=products['Provider/Company Name'].value_counts()[:10].index)
plt.title("Top 10 Ownership share of company and products")
plt.xticks(rotation=90);
for p in ax.patches:
    ax.text (p.get_x() + p.get_width()  / 2,p.get_height()+ 0.75,p.get_height(), fontsize = 11)


Google seems to have the hghest share of products used with 30 of them , Microsoft has the second most products in the market with 6 tied with Houghton Mifflin Harcourt

In [ ]:
fig = px.pie(products['Provider/Company Name'].value_counts().reset_index().rename(columns = {'Provider/Company Name': 'count'}).head(10), values = 'count', names = 'index', width = 700, height = 700)

fig.update_traces(textposition = 'inside', 
                  textinfo = 'percent + label', 
                  hole = 0.7, 
                  marker = dict(colors = ['#90afc5','#336b87','#2a3132','#763626', 'a43820'], line = dict(color = 'white', width = 2)))

fig.update_layout(annotations = [dict(text = 'Company Market Share', 
                                      x = 0.5, y = 0.5, font_size = 26, showarrow = False, 
                                      font_family = 'monospace',
                                      font_color = '#283655')],
                  showlegend = False)
                  
fig.show()

## Sector

What was the main sector of interest for the users?

In [ ]:
products['Sector(s)'].value_counts()

In [ ]:

plt.figure(figsize=(12,8))
ax=sns.countplot(products['Sector(s)'],palette='cubehelix_r',order=products['Sector(s)'].value_counts()[:10].index)
plt.title("Top 10 Sectors")
plt.xticks(rotation=90);
for p in ax.patches:
    ax.text (p.get_x() + p.get_width()  / 2,p.get_height()+ 0.75,p.get_height(), fontsize = 11)


In [ ]:
fig = px.pie(products['Sector(s)'].value_counts().reset_index().rename(columns = {'Sector(s)': 'count'}).head(10), values = 'count', names = 'index', width = 700, height = 700)

fig.update_traces(textposition = 'inside', 
                  textinfo = 'percent + label', 
                  hole = 0.7, 
                  marker = dict(colors = ['#90afc5','#336b87','#2a3132','#763626', 'a43820'], line = dict(color = 'white', width = 2)))

fig.update_layout(annotations = [dict(text = 'Sector Usage', 
                                      x = 0.5, y = 0.5, font_size = 26, showarrow = False, 
                                      font_family = 'monospace',
                                      font_color = '#283655')],
                  showlegend = False)
                  
fig.show()

Prek has the highest products under its sector, we should keep this in mind for future analysis

## Function Class

In [ ]:
products['Function Class'].value_counts()

In [ ]:

plt.figure(figsize=(12,8))
ax=sns.countplot(products['Function Class'],palette='cubehelix_r',order=products['Function Class'].value_counts().index)
plt.title("Top 10 Sectors")
plt.xticks(rotation=90);
for p in ax.patches:
    ax.text (p.get_x() + p.get_width()  / 2,p.get_height()+ 0.75,p.get_height(), fontsize = 11)


Most products fall in the LC category lets have a look at the sub-category that has the most products

In [ ]:

plt.figure(figsize=(12,8))
ax=sns.countplot(products['Function Desc'],palette='cubehelix',order=products['Function Desc'].value_counts().index)
plt.title("Function Descriptions")
plt.xticks(rotation=90);
for p in ax.patches:
    ax.text (p.get_x() + p.get_width()  / 2,p.get_height()+ 0.75,p.get_height(), fontsize = 11)


Its only fair that Digital Learning Platforms get the top spot for products used as during the pandemic most of studies happened online

### What to look for?

## Engagement Data

In [ ]:
data

lets review the data

In [ ]:
data.info()

In [ ]:
data.shape

The dataset has 2.2 Million records and 4 features that is huge for sure

In [ ]:
null_values(data)

With the size of our data hence we can't start investigating 5378409 missing values and hence for this we drop them

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.reset_index(drop=True,inplace=True)

In [ ]:
data

## Preprocessing

In [ ]:
data.info()

We can create something off the time feature so that we can have more insights genrated

In [ ]:
data['time']=pd.to_datetime(data['time'])

In [ ]:
data.info()

## Feture Generation

In [ ]:
#since the year was the same 2021 we don't require the year column
data['month'] = data['time'].dt.month
data['date']=data['time'].dt.day
data['date_week']=data['time'].dt.weekday

In [ ]:
data

## Understanding the data

### Engagement

In [ ]:
#we aggreagte the data by date to get some insights for now
date_agg=data.groupby(['date','month','date_week']).agg({'engagement_index':np.mean,'pct_access':np.mean})
date_agg

Lets get some insights on the trend

In [ ]:
date_agg=date_agg.reset_index()

In [ ]:
date_agg

### Monthly Engagement

In [ ]:
plt.figure(figsize=(15,11))
sns.lineplot(y=date_agg['pct_access'],x=date_agg["month"],palette='rocket')
plt.title("Average access per month")


Seems like there's a trend, we assume the break is due to the summer break that was there from June to Mid August

In [ ]:
plt.figure(figsize=(15,11))
sns.lineplot(y=date_agg['engagement_index'],x=date_agg["month"],palette='rocket')
plt.title("Average Engagemnet Index per month")


Seems like the first month was slow in the engagement index, lets check on that 

In [ ]:
# plt.figure(figsize=(15,11))
# sns.lineplot(y=date_agg['engagement_index'],x=date_agg["month"],palette='rocket')
# sns.lineplot(y=date_agg['pct_access'],x=date_agg["month"],palette='rocket')
# plt.title("Average Engagemnet Index per month")


In [ ]:
plt.figure(figsize=(15,11))
sns.lineplot(hue=date_agg['date_week'],y=date_agg['pct_access'],x=date_agg["month"],palette='rocket')
plt.title("Average access per month per day")


A couple of things to note
1. 0 is Monday and 6 is sunday
2. we see interaction is quite low on the weekends
3. We see a large dip in the month of July where almost all engagements went close to zero
4. Engagement reached peek in months of January and September, the trend starts dipping From february

In [ ]:
plt.figure(figsize=(15,11))
sns.lineplot(hue=date_agg['date_week'],y=date_agg['engagement_index'],x=date_agg["month"],palette='twilight_shifted')
plt.title("Average access per month per day")


Seems like we get a different graph as the one above, engament spikes up in the fourth month during which we experience the most engagment on Mondays,

However there's something interesting about Wednesdays it has a quite amusing graph. That needs to be investigated soon

### Date of the week

In [ ]:
plt.figure(figsize=(15,10))
sns.lineplot(y=date_agg['engagement_index'],x=date_agg["date_week"])

As expected there is more engagment at the begining of the week and it slowly drops as we get to the weekend , with Tuesday having the highest number of  average engagment 

In [ ]:
plt.figure(figsize=(15,10))
sns.lineplot(y=date_agg['pct_access'],x=date_agg["date_week"])

This follows a similar curve as the engagment_index

## Merge the Data

In [ ]:
finaldf=pd.merge(products,data,left_on='LP ID',right_on='lp_id')

In [ ]:
finaldf

In [ ]:
# finaldf['district_id']=finaldf['district_id'].astype('int')